In [2]:
from manim import *
from numpy import linalg as npl
import math
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline
from multiprocessing import Process

video_scene = " -v WARNING --progress_bar None --disable_caching taylor_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s taylor_Scene"

Manim Community v0.17.3

Jupyter Capture Output v0.0.8


In [3]:
# n-te Ableitung des sinus an der Stelle a
def sine_derivative(a, n):
    factor = PI / 4
    if n % 4 == 0:
        return np.sin(a*factor)
    elif n % 4 == 1:
        return np.cos(a*factor)
    elif n % 4 == 2:
        return -np.sin(a*factor)
    elif n % 4 == 3:
        return -np.cos(a*factor)
    

def e_derivative(a, n):
    return np.exp(a) /100


# Taylor-Reihe einer Funktion func(a, n) im Punkt a mit Grad n
def Taylor(func_n, n, a):
    def Taylor_n(x):
        taylor_series = 0
        for i in range(n+1):
            taylor_series += func_n(a, i)/math.factorial(i)*(PI/4*(x-a))**i
        return taylor_series
    return Taylor_n

# Einzelnes Element der Taylor-Reihe
def Taylor_Seq_Element(func_n, n, a):
    def Taylor_n(x):
        taylor_series = 0
        for i in range(n+1):
            taylor_series = func_n(a, i)/math.factorial(i)*(PI/4*(x-a))**i
        return taylor_series
    return Taylor_n


# Find x-limit for given y
def find_xlim_plus(func, y_limit, interval):
    func_down_limit = interval[0]
    func_up_limit = interval[1]
    x_array = np.linspace(func_down_limit, func_up_limit, 1000)
    x_down = interval[0]
    x_up = interval[1]
    for x in x_array:
        y = func(x)
        #print(y)
        if y <= -y_limit:
            x_down = x
        if y >= y_limit:
            x_up = x
            return [x_down, x_up]
    return [x_down, x_up]


def find_xlim_minus(func, y_limit, interval):
    func_down_limit = interval[0]
    func_up_limit = interval[1]
    x_array = np.linspace(func_down_limit, func_up_limit, 1000)
    x_down = interval[0]
    x_up = interval[1]
    for x in x_array:
        y = func(x)
        #print(y)
        if y >= y_limit:
            x_down = x
        if y <= -y_limit:
            x_up = x
            return [x_down, x_up]
    return [x_down, x_up]

In [7]:
%%capture_video --path "animations/taylor/taylor.mp4"
%%manim -qh --fps 60 $video_scene

class taylor_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        # Taylor-Headline
        text_taylor = Tex(r"Taylorreihe $Tf(x; a)=\sum\frac{f^{(n)}(a)}{n!}(x-a)^n$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
        text_taylor_formula = Tex(r"Taylorreihe $T\mathrm{sin}(x; a)=\sum\frac{\sin^{(n)}}{n!}(x-a)^n$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
        text_taylor_formula_1 = Tex(r"Taylorreihe $T_1\mathrm{sin}(x; \pi)\approx\sum_0^1\frac{\sin^{(n)}{(\pi)}}{n!}(x-\pi)^n$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
        text_taylor_formula_3 = Tex(r"Taylorreihe $T_3\mathrm{sin}(x; \pi)\approx\sum_0^3\frac{\sin^{(n)}{(\pi)}}{n!}(x-\pi)^n$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
        text_taylor_formula_5 = Tex(r"Taylorreihe $T_5\mathrm{sin}(x; \pi)\approx\sum_0^5\frac{\sin^{(n)}{(\pi)}}{n!}(x-\pi)^n$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
        text_taylor_formula_7 = Tex(r"Taylorreihe $T_7\mathrm{sin}(x; \pi)\approx\sum_0^7\frac{\sin^{(n)}{(\pi)}}{n!}(x-\pi)^n$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
        text_taylor_formula_1[0][22:43].set_color(BLUE)
        text_taylor_formula_3[0][22:43].set_color(BLUE)
        text_taylor_formula_5[0][22:43].set_color(BLUE)
        text_taylor_formula_7[0][22:43].set_color(BLUE)
        text_taylor_formula_sine = Tex(r"$\mathrm{sin}(x)\approx T\mathrm{sin}(x; a)$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(3.5 * RIGHT)


        # Main: Coordinate System
        origin = [-2.5, -0.5, 0]

        x_range = [-0.5, 3.75*PI, 1]
        y_range = [-2.25, 2.25, 1]

        x_length = 7
        y_length = 5.5

        x_dict = dict(zip([4, 8], [r"$\pi$", r"$2\pi$"]))
        y_dict = dict(zip([-1, 0, 1], [-1, 0, 1]))

        npla = NumberPlane(
            x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}, ).move_to(origin)
        ax = Axes(x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, axis_config = {"stroke_width": 1, "stroke_opacity": 0.75}).move_to(origin).add_coordinates(x_dict, y_dict)

        ax_xlabel = ax.get_x_axis_label(Tex(r"x", font_size = 36)).shift(0.25*DOWN + 0.125*RIGHT)
        ax_ylabel = ax.get_y_axis_label(Tex(r"y", font_size = 36)).shift(0.8125*LEFT + 0.5*DOWN)

        coordinate_system = VGroup(npla, ax, ax_xlabel, ax_ylabel)

        a_factor = PI/4
        sine = ax.plot(lambda x: np.sin(x*a_factor), color = WHITE, stroke_width = 1.5, x_range = [0, 3.5*PI])

        taylor_sine_1 = ax.plot(Taylor(sine_derivative, 1, 4), color = BLUE, stroke_width = 1.5, stroke_opacity = 1, x_range = find_xlim_minus(Taylor(sine_derivative, 1, 4), 2, [0, 3.5*PI]))
        taylor_sine_3 = ax.plot(Taylor(sine_derivative, 3, 4), color = BLUE, stroke_width = 1.5, stroke_opacity = 1, x_range = find_xlim_plus(Taylor(sine_derivative, 3, 4), 2, [0, 3.5*PI]))
        taylor_sine_5 = ax.plot(Taylor(sine_derivative, 5, 4), color = BLUE, stroke_width = 1.5, stroke_opacity = 1, x_range = find_xlim_minus(Taylor(sine_derivative, 5, 4), 2, [0, 3.5*PI]))
        taylor_sine_7 = ax.plot(Taylor(sine_derivative, 7, 4), color = BLUE, stroke_width = 1.5, stroke_opacity = 1, x_range = find_xlim_plus(Taylor(sine_derivative, 7, 4), 2, [0, 3.5*PI]))#, y_range = [-2, 2])


        # Corner: Coordinate System
        origin_corner = [4.25, -1.5, 0]
        rect_corner = Rectangle(width = 4, height = 2.5, stroke_width = 0.75).move_to(origin_corner)

        # Corner-Headline
        text_taylor_formula_1_corner = Tex(r"$\cos{(\pi)}\cdot(x-\pi)$", font_size = 48).move_to(origin_corner).shift(2 * UP)
        text_taylor_formula_3_corner = Tex(r"$\frac{-\cos{(\pi)}}{3!}\cdot(x-\pi)^3$", font_size = 48).move_to(origin_corner).shift(2 * UP)
        text_taylor_formula_5_corner = Tex(r"$\frac{\cos{(\pi)}}{5!}\cdot(x-\pi)^5$", font_size = 48).move_to(origin_corner).shift(2 * UP)
        text_taylor_formula_7_corner = Tex(r"$\frac{-\cos{(\pi)}}{7!}\cdot(x-\pi)^7$", font_size = 48).move_to(origin_corner).shift(2 * UP)
        text_taylor_formula_1_corner[0][0:15].set_color(RED)
        text_taylor_formula_3_corner[0][0:19].set_color(RED)
        text_taylor_formula_5_corner[0][0:19].set_color(RED)
        text_taylor_formula_7_corner[0][0:19].set_color(RED)

        x_range = [-0.5, 3.75*PI, 1]
        y_range = [-2.25, 2.25, 1]

        x_length = 6 / 2
        y_length = 4 / 2

        npla_corner = NumberPlane(
            x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}, ).move_to(origin_corner)
        ax_corner = Axes(
            x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, 
            axis_config = {"stroke_width": 1, "stroke_opacity": 0.75}, tips = False).move_to(origin_corner).add_coordinates(x_dict, y_dict, font_size = 20)

        coordinate_system_corner = VGroup(npla_corner, ax_corner, rect_corner)
        x0_dot = Circle(radius = 0.075, color = BLUE, fill_opacity = 0.5).move_to(ax.c2p(4, 0))

        taylor_sine_1_corner = ax_corner.plot(Taylor_Seq_Element(sine_derivative, 1, 4), color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = find_xlim_minus(Taylor_Seq_Element(sine_derivative, 1, 4), 2, [0, 3.5*PI]))
        taylor_sine_3_corner = ax_corner.plot(Taylor_Seq_Element(sine_derivative, 3, 4), color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = find_xlim_plus(Taylor_Seq_Element(sine_derivative, 3, 4), 2, [0, 3.5*PI]))
        taylor_sine_5_corner = ax_corner.plot(Taylor_Seq_Element(sine_derivative, 5, 4), color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = find_xlim_minus(Taylor_Seq_Element(sine_derivative, 5, 4), 2, [0, 3.5*PI]))
        taylor_sine_7_corner = ax_corner.plot(Taylor_Seq_Element(sine_derivative, 7, 4), color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = find_xlim_plus(Taylor_Seq_Element(sine_derivative, 7, 4), 2, [0, 3.5*PI]))

        # self.add(rect_corner, coordinate_system_corner)
        # self.add(text_taylor_formula_3, coordinate_system, taylor_sine_1_corner)
        # self.add(text_taylor_formula_1_corner)

        self.play(Write(text_taylor), run_time = 3)
        self.play(FadeIn(coordinate_system), run_time = 3)
        self.wait(0.5)
        self.play(Create(sine), Transform(text_taylor, text_taylor_formula), run_time = 3)
        self.wait(1.5)
        self.play(FadeIn(coordinate_system_corner), run_time = 3)
        self.wait(1.5)
        self.play(Create(taylor_sine_1), Transform(text_taylor, text_taylor_formula_1), Create(taylor_sine_1_corner), FadeIn(text_taylor_formula_1_corner), FadeIn(x0_dot), run_time = 2)
        self.wait(2)
        self.play(Transform(text_taylor, text_taylor_formula_3), Transform(taylor_sine_1, taylor_sine_3), Transform(taylor_sine_1_corner, taylor_sine_3_corner), Transform(text_taylor_formula_1_corner, text_taylor_formula_3_corner), run_time = 2)
        self.wait(2)
        self.play(Transform(text_taylor, text_taylor_formula_5), Transform(taylor_sine_1, taylor_sine_5), Transform(taylor_sine_1_corner,taylor_sine_5_corner), Transform(text_taylor_formula_1_corner, text_taylor_formula_5_corner), run_time = 2)
        self.wait(2)
        self.play(Transform(text_taylor, text_taylor_formula_7), Transform(taylor_sine_1, taylor_sine_7), Transform(taylor_sine_1_corner,taylor_sine_7_corner), Transform(text_taylor_formula_1_corner, text_taylor_formula_7_corner), run_time = 2)
        self.wait(5)

Output saved by overwring previous file at animations/taylor/taylor.mp4.
